## Analyzing a Movie Dataset

### Getting general information about the dataset

In [132]:
# Loading the dataset
import pandas as pd
import numpy as np

movies_df = pd.read_csv("imdb_top_1000.csv")

# Print first few rows of the dataset
movies_df.head()



,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [ ]:
# Get general information about the dataframe
movies_df.info()

In [ ]:
# Summary statistics of numerical columns
movies_df.describe()
# for this dataset, it doesn't tell me too much

In [ ]:
movies_df.set_index('Series_Title', inplace=True) # Change index to be Title column
movies_df.head(4)

In [ ]:
movies_df.tail(10)

In [ ]:
# Add a 'Rank' column
movies_df['Rank'] = range(1, len(movies_df) + 1)
movies_df.info()

### Cleaning up the dataset
* adding columns before analysis
* removing rows
* cleaning up strings
* converting data types

In [ ]:
# get list of column names
new_columns = list(movies_df.columns)
# remove the 'Rank' column from the list
new_columns.remove('Rank')
# insert 'Rank' column at specified index
new_columns.insert(0, 'Rank')
# Reorder the DataFrame Columns
movies_df = movies_df[new_columns]
# Check DataFrame info
movies_df.info()

In [ ]:
# Sorting the data by ratings
movies_df = movies_df.sort_values(by='IMDB_Rating', ascending=False)

movies_df.reset_index(inplace=True)

movies_df.set_index('Rank', inplace=True)

movies_df

In [126]:
# find the row by column
movies_df.shape

(1000, 17)

In [127]:
movies_df.size

17000

In [ ]:
movies_df.sort_index().head()

In [ ]:
# Handling missing data
print(movies_df.isnull().sum())

In [ ]:
#movies_df.fillna(0)
#print(movies_df.isna().sum())
#lets remove rows with null data
movies_df_clean = movies_df.dropna()
movies_df_clean.info()

In [ ]:
# determine if there is any duplicated data
sum(movies_df.duplicated())

In [ ]:
# convert gross to numeric type coerce errors to NaN for non numeric values
movies_df_clean['Gross'] = pd.to_numeric(movies_df_clean['Gross'], errors='coerce')
movies_df_clean.info()

In [ ]:
movies_df_clean['Runtime'] = movies_df_clean['Runtime'].str.replace(' min', '')
# Convert the 'Runtime' column to numeric type
movies_df_clean['Runtime'] = pd.to_numeric(movies_df_clean['Runtime'], errors='coerce')
movies_df_clean.head()

In [ ]:
print(movies_df_clean['Runtime'].dtype) # checking runtime datatype

In [ ]:
# drop rows with missing gross data
movies_df.dropna(subset=['Gross'], inplace=True)
movies_df

In [ ]:
movies_df['Released_Year'] = pd.to_datetime(movies_df['Released_Year'])
movies_df.head()
# doesnt work because dont have the full date only the year

In [133]:
# Create a mask to identify rows with valid years (to remove rows with invalid year values)
valid_year_mask = movies_df['Released_Year'].str.match(r'^\d{4}$')

# Filter out rows with invalid years
movies_df = movies_df[valid_year_mask]

# Convert 'Released_Year' to datetime
movies_df['Released_Year'] = pd.to_datetime(movies_df['Released_Year'] + '-01-01')

# Print the updated DataFrame
movies_df.head()


,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994-01-01,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972-01-01,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008-01-01,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974-01-01,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957-01-01,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [ ]:
movies_df['Released_Year'] = pd.to_datetime(movies_df['Released_Year'])
movies_df['Released_Year'] = movies_df['Released_Year'].dt.year

In [ ]:
# drop unused columns
movies_df.drop(['Certificate', 'Poster_Link'], axis=1, inplace=True)
movies_df.shape

In [ ]:
# String cleaning
movies_df['Genre'] = movies_df['Genre'].str.strip().str.lower()
print(movies_df['Genre'].head())

In [ ]:
# correcting data types
movies_df['Gross'] = movies_df['Gross'].str.replace(',', '').astype(float)
movies_df

### Analyzing the data

In [135]:
# Get genre count of top 5 most common genre
movies_df['Genre'].value_counts().head(5)

Genre
Drama                     85
Drama, Romance            37
Comedy, Drama             35
Comedy, Drama, Romance    31
Action, Crime, Drama      30
Name: count, dtype: int64

In [ ]:
# Show only movies with genre name Drama
movies_df[movies_df['Genre'] == 'Drama']

In [137]:
# Get movies with released after 2015
cutoff_date = pd.to_datetime('2015-01-01')
after_year = movies_df['Released_Year'] > cutoff_date
movies_df[after_year].head(5) #display only 5 rows

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
18,https://m.media-amazon.com/images/M/MV5BNjViNW...,Hamilton,2020-01-01,PG-13,160 min,"Biography, Drama, History",8.6,The real life of one of America's foremost fou...,90.0,Thomas Kail,Lin-Manuel Miranda,Phillipa Soo,Leslie Odom Jr.,Renée Elise Goldsberry,55291,NaN
19,https://m.media-amazon.com/images/M/MV5BYWZjMj...,Gisaengchung,2019-01-01,A,132 min,"Comedy, Drama, Thriller",8.6,Greed and class discrimination threaten the ne...,96.0,Bong Joon Ho,Kang-ho Song,Lee Sun-kyun,Cho Yeo-jeong,Choi Woo-sik,552778,"53,367,844"
20,https://m.media-amazon.com/images/M/MV5BOTc2ZT...,Soorarai Pottru,2020-01-01,U,153 min,Drama,8.6,"Nedumaaran Rajangam ""Maara"" sets out to make t...",NaN,Sudha Kongara,Suriya,Madhavan,Paresh Rawal,Aparna Balamurali,54995,NaN
33,https://m.media-amazon.com/images/M/MV5BNGVjNW...,Joker,2019-01-01,A,122 min,"Crime, Drama, Thriller",8.5,"In Gotham City, mentally troubled comedian Art...",59.0,Todd Phillips,Joaquin Phoenix,Robert De Niro,Zazie Beetz,Frances Conroy,939252,"335,451,311"
53,https://m.media-amazon.com/images/M/MV5BMmExNz...,Capharnaüm,2018-01-01,A,126 min,Drama,8.4,While serving a five-year sentence for a viole...,75.0,Nadine Labaki,Zain Al Rafeea,Yordanos Shiferaw,Boluwatife Treasure Bankole,Kawsar Al Haddad,62635,"1,661,096"


In [ ]:
#movie with highest votes
# Sort the DataFrame by 'Number of Votes' column in descending order
movies_sorted_by_votes = movies_df_clean.sort_values(by='No_of_Votes', ascending=False)

# Get the top 5 movies with the most votes
top_movies_by_votes = movies_sorted_by_votes.head(5)

# Print the top movies with the most votes
print("Top 5 movies with the most votes:")
display(top_movies_by_votes)



In [ ]:
#which movies have the longest runtime
# Find the movie with the longest runtime
movie_longest_runtime = movies_df_clean.nlargest(1, 'Runtime')

# Print the movie details
print("Movie with the longest runtime:")
movie_longest_runtime


In [ ]:
#movie with shortest runtime
# Find the movie with the shortest runtime
movie_shortest_runtime = movies_df_clean.nsmallest(1, 'Runtime')

# Print the movie details
print("Movie with the shortest runtime:")
movie_shortest_runtime


In [134]:
# determine which year had the most released movies
best_year = movies_df.groupby('Released_Year').count()['Series_Title']
#best_year.shape
pd.set_option('display.max_rows', None)
best_year
#pd.reset_option('diplay.max_rows')

Released_Year
1920-01-01     1
1921-01-01     1
1922-01-01     1
1924-01-01     1
1925-01-01     2
1926-01-01     1
1927-01-01     2
1928-01-01     2
1930-01-01     1
1931-01-01     3
1932-01-01     2
1933-01-01     3
1934-01-01     2
1935-01-01     3
1936-01-01     1
1937-01-01     1
1938-01-01     3
1939-01-01     5
1940-01-01     7
1941-01-01     2
1942-01-01     3
1943-01-01     1
1944-01-01     4
1945-01-01     2
1946-01-01     5
1947-01-01     2
1948-01-01     6
1949-01-01     3
1950-01-01     5
1951-01-01     5
1952-01-01     4
1953-01-01     5
1954-01-01     6
1955-01-01     6
1956-01-01     5
1957-01-01     9
1958-01-01     4
1959-01-01     7
1960-01-01    11
1961-01-01     5
1962-01-01    13
1963-01-01     5
1964-01-01     7
1965-01-01     4
1966-01-01     7
1967-01-01    10
1968-01-01     8
1969-01-01     3
1970-01-01     3
1971-01-01     9
1972-01-01     8
1973-01-01    12
1974-01-01     6
1975-01-01     9
1976-01-01     7
1977-01-01     3
1978-01-01     7
1979-01-01    12


In [ ]:
# which movies had the hishgest gross
max_gross_index = movies_df['Gross'].idxmax() #get index of movie with highest gross
movie_with_highest_gross = movies_df.loc[max_gross_index, 'Series_Title']
print(movie_with_highest_gross)

In [ ]:
# Grouping the movies and looking at 
genre_group = movies_df.groupby('Genre').agg({'IMDB_Rating': 'mean', 'Gross': 'sum'})
print(genre_group)

### Merging Files example

In [ ]:
# loading new datasets
df1 = pd.read_csv("tmdb_5000_credits.csv")
df2 = pd.read_csv("tmdb_5000_movies.csv")
df1.info()

In [ ]:
df2.info()

In [ ]:
df1.columns = ['id', 'title', 'cast', 'crew']
df2 = df2.merge(df1, on='id')
df2.info()

In [ ]:
# Look at the new data
df2.head(5)

In [ ]:
# look at mean voting average
c = df2['vote_average'].mean()
c
#so most movies in this list is approx a 6 out of 10 rating

In [ ]:
# lets get only movies that have a certain number of votes so movies with only 3 votes aren't included
m = df2['vote_count'].quantile(0.9)
qualify_movies = df2.copy().loc[df2['vote_count'] >= m] # make a new DataFrame
qualify_movies.shape #481 movies out of the 5000 movies qualify

In [ ]:
def weighted_rating(x, m=m, c=c):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * c)

qualify_movies['score'] = qualify_movies.apply(weighted_rating, axis=1)
qualify_movies = qualify_movies.sort_values('score', ascending=False)
qualify_movies[['title', 'vote_count', 'vote_average', 'score']].head(10)